# Creating a recommender system from scratch

Part 2 of 2

*Written by: James Soh*

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse.linalg import svds

%matplotlib inline
np.random.seed(0)
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
from sklearn.utils import shuffle

In [4]:
movies_df = pd.read_csv('./ml-latest-small/movies.csv', names=['MovieID', 'Title', 'Genres'], header=0)
ratings_df = pd.read_csv('./ml-latest-small/ratings.csv', names=['UserID', 'MovieID', 'Rating', 'Timestamp'], header=0)
tags_df = pd.read_csv('./ml-latest-small/tags.csv', names=['UserID', 'MovieID', 'tag', 'Timestamp'], header=0)
links_df = pd.read_csv('./ml-latest-small/links.csv', names=['MovieID', 'imdbId', 'tmbdId'], header=0)

In [5]:
links_df = pd.read_csv('./ml-latest-small/links.csv', names=['MovieID', 'imdbId', 'tmbdId'], header=0)

### Train-test split and utilizing implicit data (timestamp)

I chose a train ratio of 0.7 so as not to overfit.

Each user's ratings has a timestamp attached to it. I will set 30% of each user's latest ratings as the test set. Because we want to make predictions for a user's future preferences, fitting our model for the user's latest ratings will give the best predictions for a user's future preferences.



In [6]:
def train_test_split(data, train_ratio):
    '''Creates a train/test split from a pandas DataFrame that contains rating data. The dataframe should have
    columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']. For each user, items are sorted by timestamp before sending to
    test set. Hence test set will contain users latest ratings.
    
    Args:
        data: Ratings dataframe
        train_ratio: float between 0 and 1
    
    '''
    
    
    train = pd.DataFrame(columns=data.columns)
    test = pd.DataFrame(columns=data.columns)
    users = list(set(data['UserID']))
    dummy_movieid = list(ratings_df.MovieID.unique()) #To ensure our matrix later contains all MovieID.
    dummy_movieid = pd.DataFrame({'UserID':999, 'MovieID':dummy_movieid, 'Rating':0, 'Timestamp':0})
    for u in users:
        temp = data[data['UserID'] == u]
        n = len(temp)
        test_size = int(train_ratio*n)
        temp = temp.sort_values('Timestamp', ascending=False).reset_index(drop=True)
        dummy_train = temp.iloc[:test_size]
        dummy_test = temp.iloc[test_size:]
        train = pd.concat([train, dummy_train])
        test = pd.concat([test, dummy_test])
    train = train.append(dummy_movieid)
    test = test.append(dummy_movieid)

    
    return train, test
    

In [7]:
train, test = train_test_split(ratings_df, 0.5)

In [8]:
print train.shape
print test.shape

(58904, 4)
(59232, 4)


### Factorizing the matrix and minimizing error with gradient descent.

Given set of U users, set of D items. R is the matrix of size U x D. We want to discover K latent features. 

Our task is to find 2 smaller matrices P and Q that best approximates R.

P represents user feature vectors, and Q represents item feature vectors. (Or, how much a user likes a certain concept, and how much a certain concept is represented in a movie.)


\begin{equation*}
\mathbf{R} \approx \mathbf{P} \times \mathbf{Q}^T = \hat{\mathbf{R}}
\end{equation*}  

To get the prediction of an item in the matrix, calculate dot product of the 2 corrosponding vectors.


\begin{equation*} 
\hat{r}_{ij} = p_i^T q_j = \sum_{k=1}^k{p_{ik}q_{kj}}
\end{equation*}

Now, initialize P and Q with random values, calculate how different the product is to matrix R, and try to reduce the difference iteratively.


\begin{equation*} 
e_{ij}^2 = (r_{ij} - \hat{r}_{ij})^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2
\end{equation*}

Need to find the gradient to know which direction to modify P(i,k) and Q(k,j)



\begin{equation*}
\frac{\partial}{\partial p_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(q_{kj}) = -2 e_{ij} q_{kj}
\end{equation*}


\begin{equation*}
\frac{\partial}{\partial q_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(p_{ik}) = -2 e_{ij} p_{ik}
\end{equation*}


Now, we can form our update rules. Also attach a learning rate.
\begin{equation*}
p'_{ik} = p_{ik} + \alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + 2\alpha e_{ij} q_{kj} 
\end{equation*}

\begin{equation*}
q'_{kj} = q_{kj} + \alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + 2\alpha e_{ij} p_{ik} 
\end{equation*}

I am only reducing the errors of known item:value pairs. Our latent vectors will be discovered, and then we can form predictions on unkown items. 

In [9]:
 def initialize_random_matrices(ratings, K):
        '''
        Returns 2 matrices of size N,K and M,K that contains random floats from 0 to 1. 

        Args:
            ratings: N by M matrix containing ratings. Rows are users, columns are products.
            K: Number of latent factors to discover
            
        '''
        R = np.array(ratings)
        N = len(R) 
        M = len(R[0])  
        P = np.random.random((N,K))
        Q = np.random.random((M,K)) #Note: We will transpose this matrix later.
        return P, Q


### Quick demonstration of the GD in action. 

this demonstration I've set the learning rate to be quite high (0.01) with a small amount of steps (5), and to print the product of the 2 smaller matrixes. Let's observe how the product of the component matrices converge towards the original matrix.

In [10]:
def matrix_factorization_test(R, P, Q, K, steps=1000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)): # iterate over rows (or users)
            for j in xrange(len(R[i])): # for each user, iterate all items. 30 times.
                #Hence, iterate over entire row of matrix R, before moving on to next row.
                if R[i][j] > 0: # If a given element in a matrix is > 0, aka user has actually rated an item
                    # eij is the error for a given element in matrix R
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j]) 
                    # Iterate over the user:feature matrix.
                    for k in xrange(K): # K is number of latent features. 
                        # Updating each indivudual feature of the matrix P and matrix Q. 
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])): #for elements in the original matrix:
                if R[i][j] > 0: #if element > 0 (aka if a rating is there):
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2) #sum up the square of the error
                    for k in xrange(K): 
                        #for each element in R matrx, iterate K also
                            e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2) )
        print '=============== Step %s =============' % step
        print np.dot(P,Q)
        print 'Total squared error for step %s' %step
        print e
        if e < 0.001:
            break
    return P, Q.T



In [11]:
np.random.seed(1)
test_R = np.random.randint(0,6, (6,6))
test_P, test_Q = initialize_random_matrices(test_R, 2)
print 'This is the ground truth matrix'
print test_R

test_P, test_Q = matrix_factorization_test(test_R, test_P, test_Q, 2, steps=10, alpha=0.005)
matrix_factorization_test(test_R, test_P, test_Q, 2, steps=10, alpha=0.005)



This is the ground truth matrix
[[5 3 4 0 1 3]
 [5 0 0 1 4 5]
 [4 1 2 4 5 2]
 [4 3 4 2 4 5]
 [2 4 1 1 0 5]
 [1 1 5 1 1 0]]
=============== Step 0 =============
[[ 0.92581623  0.91391792  0.85486489  0.83337782  0.62720094  0.43445172]
 [ 0.1325813   0.32460568  0.17891314  0.13082131  0.27331743  0.13289893]
 [ 0.91698515  0.93080109  0.85417594  0.82694522  0.64546728  0.43964826]
 [ 0.58391647  1.48200305  0.80324326  0.57926779  1.25335346  0.60442304]
 [ 0.61312005  1.23147463  0.74874657  0.58900475  1.0085312   0.5162016 ]
 [ 0.39797065  1.01399147  0.54859864  0.39503494  0.8579471   0.41337951]]
Total squared error for step 0
245.993198789
=============== Step 1 =============
[[ 1.14257885  1.09204038  1.04633081  0.96937728  0.80527588  0.62336351]
 [ 0.26738426  0.48086411  0.31476665  0.23406391  0.4066785   0.24455602]
 [ 1.13885757  1.12312104  1.05366996  0.96732883  0.83620245  0.63650338]
 [ 0.84666381  1.72496194  1.05947299  0.7476322   1.4837001   0.86298939]
 [ 0.79

(array([[ 1.83403387,  0.41781682],
        [ 1.56070215,  1.27055253],
        [ 1.69089255,  0.47663879],
        [ 1.43471259,  1.63562729],
        [ 0.90363501,  1.38260931],
        [ 0.74291519,  0.80782788]]), array([[ 2.12206343,  0.64310253],
        [ 0.94857244,  1.2635675 ],
        [ 1.54431835,  0.96871409],
        [ 1.27292475,  0.16944054],
        [ 1.22247141,  1.38101932],
        [ 1.48532861,  1.63799612]]))

### Testing pickle

In [12]:
test_P_df = pd.DataFrame(test_P)
test_P_df.to_pickle('test_P.pkl')

In [13]:
def matrix_factorization(R, P, Q, K, steps=1000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)): # iterate over rows (or users)
            for j in xrange(len(R[i])): # for each user, iterate all items. 30 times.
                #Hence, iterate over entire row of matrix R, before moving on to next row.
                if R[i][j] > 0: # If a given element in a matrix is > 0, aka user has actually rated an item
                    # eij is the error for a given element in matrix R
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j]) 
                    # Iterate over the user:feature matrix.
                    for k in xrange(K): # K is number of latent features. 
                        # Updating each indivudual feature of the matrix P and matrix Q. 
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])): #for elements in the original matrix:
                if R[i][j] > 0: #if element > 0 (aka if a rating is there):
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2) #sum up the square of the error
                    for k in xrange(K): 
                        #for each element in R matrx, iterate K also
                            e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T



# Experimentation and Findings


1. With 20 latent factors, steps=500, alpha=0.002. Rmse is 1.0118604462. Runtime 120 minutes. 

2. With 40 latent factors, steps=500, alpha=0.002. Rmse is 1.11010187762. Runtime 192 minutes.

3. First de-meaning the data, then running experiment 1.

Huge improvement from cosine similarity and SVD! (In RMSE, not runtime.)

### Preparing train/test matrices

In [14]:
import timeit
start_time = timeit.default_timer()

train, test = train_test_split(ratings_df, 0.7)

train_matrix = train.pivot(index='UserID', columns = 'MovieID', values='Rating')
test_matrix = test.pivot(index='UserID', columns = "MovieID", values='Rating')
train_matrix.drop(999, inplace=True)
test_matrix.drop(999, inplace=True)
test_matrix.fillna(0,inplace=True)
train_matrix.fillna(0, inplace=True)



train_matrix = np.array(train_matrix)
test_matrix = np.array(test_matrix)

ground_truth = ratings_df.pivot(index='UserID', columns='MovieID', values='Rating')


elapsed = timeit.default_timer() - start_time

In [13]:
import timeit

P, Q = initialize_random_matrices(train_matrix, 20)

print P.shape
print Q.shape

(671, 20)
(9066, 20)


### <span style="color:red">Load the pickled result, or run the cell (running the cell will take about 120 minutes)</span>


In [99]:
result_P = pd.read_pickle('result_P_df.pkl')
result_P = np.array(result_P)

result_Q = pd.read_pickle('result_q_df.pkl')
result_Q = np.array(result_Q)

In [18]:
# This cells takes a 2 hours to run.

start_time = timeit.default_timer()


result_P, result_Q = matrix_factorization(train_matrix, P, Q, 20, steps=500)


elapsed = timeit.default_timer() - start_time
print elapsed

9301.90666658




### <span style="color:red">This cell is to create pickle of P and Q matrices. Do not need to run.</span>


In [101]:
result_P_df = pd.DataFrame(result_P)
result_Q_df = pd.DataFrame(result_Q)

result_P_df.to_pickle('result_P_df.pkl')
result_Q_df.to_pickle('result_q_df.pkl')

NameError: name 'result_P_df' is not defined

In [100]:
result_P

array([[ 0.18185038,  0.81896762,  0.42939372, ..., -0.0979881 ,
         0.4433499 , -0.33576179],
       [ 0.75219213,  0.49994733,  0.16029375, ..., -0.15039515,
         0.69359448,  0.61478643],
       [ 0.46303588, -0.34371818,  0.28892853, ...,  0.59892686,
        -0.1375437 ,  0.49941571],
       ..., 
       [ 0.17701386,  0.36887833,  0.63718673, ...,  0.06686219,
         0.512445  ,  0.37392249],
       [ 0.60310777,  0.45863913,  0.35570067, ..., -0.3366204 ,
         0.05125785,  0.82396467],
       [ 0.31844834,  0.81056969,  0.43695203, ...,  0.26658399,
         0.29761829,  0.83935853]])

### Getting results by multipling P and Q.T

In [31]:
print result_P.shape
print result_Q.shape
result_Q = result_Q.T
print result_Q.shape

(671, 20)
(9066, 20)
(20, 9066)


In [32]:
result = np.dot(result_P, result_Q)

In [128]:
pd.DataFrame(result).to_pickle('results_GD.pkl')


In [137]:
movies_df.to_pickle('mvies_df.pkl')
ratings_df.to_pickle('ratings_df.pkl')

### Getting the RMSE of our predictions.

In [68]:
ground_truth_list = []
result_list = []

ground_truth_array = np.array(ground_truth)
result_array = np.array(result)

i, j = np.where(test_matrix != 0)
locators = zip(i,j)

for i in locators:
    ground_truth_list.append(ground_truth_array[i])
    result_list.append(result_array[i])

ground_truth_array = np.array(ground_truth_list)
result_array = np.array(result_list)

rmse = np.sqrt(sum((ground_truth_array - result_array) ** 2) / len(ground_truth_array))

print 'Rmse is %s ' % rmse





                                              
    

Rmse is 1.0118604462 


### Experiment 2:  40 latent factors

<span style="color:blue">This cell takes about 180 minutes to run.</span>

In [26]:
P_40, Q_40 = initialize_random_matrices(train_matrix, 40)



In [30]:
print P_40.shape
print Q_40.shape

(671, 40)
(9066, 40)


In [32]:
start_time = timeit.default_timer()


result_P_40, result_Q_40 = matrix_factorization(train_matrix, P_40, Q_40, 40, steps=500)

elapsed = timeit.default_timer() - start_time
print elapsed





11575.6989019


In [35]:
result_40 = np.dot(result_P_40, result_Q_40.T)
result_40

array([[ 3.28080323,  2.75795705,  2.26646699, ...,  2.40537265,
         2.97339885,  3.22176373],
       [ 3.72343195,  3.08853744,  2.78249104, ...,  3.87448963,
         4.34606285,  3.85102228],
       [ 4.62208284,  2.91481741,  4.20128246, ...,  4.61615354,
         3.88740339,  3.46705855],
       ..., 
       [ 3.81787055,  3.83193719,  3.49064371, ...,  3.16196186,
         3.09550821,  4.76899905],
       [ 4.13851594,  3.52177309,  3.50858171, ...,  4.6557265 ,
         3.58306072,  3.02560616],
       [ 4.84760756,  3.98397751,  3.11935125, ...,  3.29496466,
         3.75185728,  3.36249914]])

In [36]:
ground_truth_list = []
result_list = []

ground_truth_array = np.array(ground_truth)
result_array = np.array(result_40)

i, j = np.where(test_matrix != 0)
locators = zip(i,j)

for i in locators:
    ground_truth_list.append(ground_truth_array[i])
    result_list.append(result_array[i])

ground_truth_array = np.array(ground_truth_list)
result_array = np.array(result_list)

rmse = np.sqrt(sum((ground_truth_array - result_array) ** 2) / len(ground_truth_array))

print 'Rmse is %s ' % rmse

Rmse is 1.11010187762 
